In [114]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import scipy.spatial.distance as dis
from scipy.ndimage import filters
import matplotlib.pyplot as plt
from io import StringIO
import cProfile
import os
import logging
import glob

In [115]:
def parsedetail(read_data_graph):
# CostCodeDefault          SESEXP => LESEQK
# CostCodeDefaultAgency    SESXR1 => LESEQY

    i=0

    for index, row in read_data_graph.iterrows():
        detail = row['detail']
        i1 = detail.count("CostCodeDefault")
        i2 = detail.count(os.linesep)+1
        if i1==i2 :
            i+=1
            
    df = read_data_graph['root']
    df = df.append (read_data_graph['child'])
    
    return df.nunique(), i
            

In [116]:
def main():
    
    os.chdir(HOME_FOLDER)
    extension = 'csv'
    files = glob.glob(INPUT_FOLDER+'*.{}'.format(extension))
    
    logger.debug(files)
    
    rpt = pd.DataFrame()
    
    for file in files:
        read_data_graph = pd.read_csv(file, encoding='utf-8', sep=',', engine='python', header=0).fillna(0)
        ids = os.path.basename(file).split("_")
        i, j = parsedetail(read_data_graph)
        saving2 = pd.DataFrame([[ids[0],ids[1],i,j]])
        rpt = rpt.append(saving2)
    
    rpt.columns=['SKUGroupID','RoomTypeID','RatePlanCount','CodeCount']
 
    top15 = rpt.groupby('SKUGroupID')['RatePlanCount'].sum().nlargest(15)
    
    rpt=rpt[rpt['SKUGroupID'].isin(top15.index)]
    
    print(rpt)
    
    print ('Sampled {} Hotels, {} Rooms, {} Rateplans'.format(rpt['SKUGroupID'].nunique(),rpt['RoomTypeID'].nunique(),rpt['RatePlanCount'].sum()))
    print ('Total {} ({:.1%}) RatePlan have code change only'.format(rpt['CodeCount'].sum(),rpt['CodeCount'].sum()/rpt['RatePlanCount'].sum()))
    
    rpt.to_csv((OUTPUT_FOLDER+'stats.csv'))


# end of main

logger = logging.getLogger()
#     CRITICAL
#     ERROR
#     WARNING
#     INFO
#     DEBUG
logging.disable(logging.DEBUG);

HOME_FOLDER = '/Users/xyao/Library/Mobile Documents/com~apple~CloudDocs/JupyterHome/Simplification/'
INPUT_FOLDER = './Result/DAG/'
OUTPUT_FOLDER = './Result/MINE/'

if __name__ == '__main__':
    main()

   SKUGroupID RoomTypeID  RatePlanCount  CodeCount
0     6416530  202379601             13          4
0    45976006  216150366             14          5
0     1155964  200260215             14          3
0      281536  216035394             15          6
0      281536  216035384             15          6
..        ...        ...            ...        ...
0     6257605  201135599             12          2
0     6257605  201135589             12          2
0      281536  216035381             15          6
0     6257605  200311359             12          2
0       12079  200263013             22         14

[269 rows x 4 columns]
Sampled 15 Hotels, 269 Rooms, 4647 Rateplans
Total 2106 (45.3%) RatePlan have code change only
